## Paso 1: Cargar los datos
En este paso, cargamos el dataset "spambase" desde un archivo. El dataset no contiene nombres de columnas, por lo que cargamos los datos sin encabezado. Luego, mostramos las primeras filas para revisar la estructura de los datos.



In [3]:
# Importar bibliotecas necesarias
import pandas as pd

# Cargar el dataset desde la ubicación local
file_path = 'https://raw.githubusercontent.com/PickLuck7/problema2/main/spambase.data?token=GHSAT0AAAAAACXFVZAHYLP2OJYMJ23SHC2AZXCHLHA'

# Cargar el archivo CSV sin encabezado
df = pd.read_csv(file_path, header=None)

# Mostrar las primeras filas para revisar los datos
df.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


## Paso 2: Preprocesamiento de los datos
Aquí separamos las características (X) y la variable objetivo (y). Las características incluyen información sobre el contenido de los correos, mientras que la variable objetivo indica si el correo es spam (1) o no spam (0). Después de esto, dividimos el dataset en dos partes: un conjunto de entrenamiento y un conjunto de prueba (80% para entrenar y 20% para probar). Además, estandarizamos las características para que todas tengan una escala similar, lo cual es importante para mejorar el rendimiento de algunos modelos.



In [4]:
# Separar características (X) y la variable objetivo (y)
X = df.iloc[:, :-1]  # Todas las columnas excepto la última
y = df.iloc[:, -1]   # La última columna (1 para spam, 0 para no spam)

# Dividir los datos en conjunto de entrenamiento y prueba (80% para entrenar, 20% para probar)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar las características
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Paso 3: Entrenamiento del modelo (Regresión Logística)
En este paso, entrenamos un modelo de regresión logística. La regresión logística es un modelo de clasificación simple pero efectivo para problemas binarios como este (donde queremos predecir si un correo es spam o no). Usamos el conjunto de entrenamiento estandarizado para ajustar el modelo.



In [5]:
# Importar modelo de regresión logística
from sklearn.linear_model import LogisticRegression

# Inicializar el modelo
model = LogisticRegression(max_iter=1000)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

## Paso 4: Realizar predicciones y evaluar el modelo
Después de entrenar el modelo, lo probamos con los datos de prueba para ver cómo de bien predice si un correo es spam. Calculamos varias métricas de evaluación:
1. **Exactitud**: Proporción de predicciones correctas.
2. **Matriz de confusión**: Muestra los verdaderos positivos, verdaderos negativos, falsos positivos y falsos negativos.
3. **Reporte de clasificación**: Incluye precisión, recall y F1-score.
4. **F1-Score**: Una métrica clave que equilibra la precisión y el recall, útil cuando los falsos positivos o negativos tienen un costo elevado.



In [6]:
# Realizar predicciones en los datos de prueba
y_pred = model.predict(X_test_scaled)

# Calcular la exactitud
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Exactitud: {accuracy * 100:.2f}%")

# Generar la matriz de confusión
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(conf_matrix)

# Generar reporte de clasificación con precisión, recall y F1-Score
from sklearn.metrics import classification_report
class_report = classification_report(y_test, y_pred)
print("Reporte de Clasificación:")
print(class_report)

# Calcular el F1-Score directamente
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred)
print(f"F1-Score: {f1:.4f}")

Exactitud: 91.97%
Matriz de Confusión:
[[506  25]
 [ 49 341]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       531
           1       0.93      0.87      0.90       390

    accuracy                           0.92       921
   macro avg       0.92      0.91      0.92       921
weighted avg       0.92      0.92      0.92       921

F1-Score: 0.9021


## Paso 5: Identificar características importantes
La regresión logística nos permite analizar los coeficientes asociados a cada característica para determinar cuáles son más importantes para predecir si un correo es spam. En este paso, mostramos las 10 características más importantes según el modelo.



In [7]:
# Identificar las características más importantes según el modelo de regresión logística
import numpy as np

# Obtener los coeficientes del modelo entrenado
feature_importance = np.abs(model.coef_[0])

# Ordenar las características por importancia
indices = np.argsort(feature_importance)[::-1]

# Mostrar las 10 características más importantes
print("Características más importantes para predecir Spam:")
for i in range(10):
    print(f"Característica {indices[i]}: Importancia = {feature_importance[indices[i]]:.4f}")

Características más importantes para predecir Spam:
Característica 26: Importancia = 4.4725
Característica 24: Importancia = 2.7965
Característica 40: Importancia = 1.6868
Característica 43: Importancia = 1.4818
Característica 41: Importancia = 1.2814
Característica 52: Importancia = 1.2340
Característica 45: Importancia = 1.1558
Característica 47: Importancia = 1.0177
Característica 28: Importancia = 0.9912
Característica 22: Importancia = 0.9840
